## Predict

Model Garden 에서 Gemma를 선택하고 deploy 버튼을 눌러 배포 

In [ ]:
from typing import Dict, List, Union

from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json

def predict_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
) -> str:
    """
    `instances` can be either single instance of type dict or a list
    of instances.
    """
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

    # The format of each instance should conform to the deployed model's prediction input schema.
    instances = instances if isinstance(instances, list) else [instances]
    instances = [
        json_format.ParseDict(instance_dict, Value()) for instance_dict in instances
    ]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )

    # The predictions are a google.protobuf.Value representation of the model's predictions.
    prediction = response.predictions[0]
    print(prediction)

    return prediction

In [48]:
prompt = "자동차는 무엇인가요?"  # @param {type: "string"}
max_tokens = 100  # @param {type: "integer"}
temperature = 1.0  # @param {type: "number"}
top_p = 1.0  # @param {type: "number"}
top_k = 10  # @param {type: "integer"}
instances = [
    {
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
    },
]

In [63]:
import time
st = time.time()

result = predict_custom_trained_model_sample(
    project="579186738071",
    endpoint_id="4374195904874283008",
    location="asia-northeast3",
    instances=instances,
    api_endpoint="asia-northeast3-aiplatform.googleapis.com"
)

et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Prompt:
자동차는 무엇인가요?
Output:
**자동차**는 엔진과 변속기, 그리고 타이어를 포함하여 자동적으로 운전할 수 있는 차량입니다. 엔진은 기름, 디젤, 전기 등을 연료로 사용하여


In [65]:
print(result.splitlines()[3])

**자동차**는 엔진과 변속기, 그리고 타이어를 포함하여 자동적으로 운전할 수 있는 차량입니다. 엔진은 기름, 디젤, 전기 등을 연료로 사용하여
